# Twitter Spam Classifier

## Dataset - Social honeypot icwsm 2011

In [2]:
#!curl -O 'http://infolab.tamu.edu/static/users/kyumin/social_honeypot_icwsm_2011.zip'&& unzip social_honeypot_icwsm_2011.zip

In [3]:
import pandas as pd
import numpy as np
import sklearn